# Banco de dados Solo Criado: Take 1

Vou fazer primeiro no notebook para ir testando, depois passo para o arquivo .py

## Roadmap

* Coletar listagem do site da prefeitura
* Transformar pdf da listagem em xmls por este site
* Transformar xmls em cvs
* Carregar cvs de listagem para o banco de dados
  * Extração;
  * Exploração;
  * Limpeza;
  * Agregação;
  * Visualização;
  * Storytelling.

## Primeira parte: Coleta e transformação dos dados

from [here](https://www.edureka.co/blog/web-scraping-with-python/): <br>
Libraries used for Web Scraping  <br>
As we know, Python is has various applications and there are different libraries for different purposes. In our further demonstration, we will be using the following libraries: <br>
<br>
**Selenium**:  Selenium is a web testing library. It is used to automate browser activities. <br>
**BeautifulSoup**: Beautiful Soup is a Python package for parsing HTML and XML documents. It creates parse trees that is helpful to extract the data easily.<br>
Pandas: Pandas is a library used for data manipulation and analysis. It is used to extract the data and store it in the desired format. 

In [20]:
import requests as rq

url = 'http://www2.portoalegre.rs.gov.br/edificapoa/default.php?p_secao=1445'
r = rq.get(url, allow_redirects=True)
open('estoques.html', 'wb').write(r.content)

87926

Se eu conseguir baixar a pagina, posso usar o BeautifulSoup para separar os links de download das listagens, e comparar com os links já armazenados talvez

## Segunda parte: a criação do banco de dados